In [147]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: ANYWIDGET_HMR=1


In [155]:
import json
from IPython.display import GeoJSON
from solvis_graphql_api.color_scale import ColourScaleNormaliseEnum, get_colour_values, get_colour_scale

In [149]:
from map_3d_widget import show_3d

In [184]:
#with open('bruceNewCrustal.geojson') as json_file:
#    bruce_crustal = json.load(json_file)
#    json_file.close()

#with open('puysegur_polys.geojson') as json_file:
#    opensha_puysegur = json.load(json_file)
#    json_file.close()

#with open('hikurangi_polys.geojson') as json_file:
#    opensha_hikurangi = json.load(json_file)
#    json_file.close()

#with open('bruceNewPuysegur.geojson') as json_file:
#    bruce_puysegur = json.load(json_file)
#    json_file.close()

#with open('bruceNewHikurangi.geojson') as json_file:
#    bruce_hikurangi = json.load(json_file)
#    json_file.close()

with open('bruce5883participation.geojson') as json_file:
    participation = json.load(json_file)
    json_file.close()

with open('bruce5883participationCrustal.geojson') as json_file:
    participationCrustal = json.load(json_file)
    json_file.close()

with open('BruceRuptures5883.json') as json_file:
    ruptures = json.load(json_file)
    json_file.close()

with open('BruceRupturesOnly5883.json') as json_file:
    ruptures_only = json.load(json_file)
    json_file.close()



In [195]:
def barchart_map(gj, property, height = 500000.0, color_scale = "inferno"):
    values = [feature["properties"][property] for feature in gj["features"]]
    color_values = get_colour_values(
                    color_scale=color_scale,
                    color_scale_vmax=max(values),
                    color_scale_vmin=max(min(values), 1e-20),
                    color_scale_normalise= ColourScaleNormaliseEnum.LIN.value,
                    values=tuple(values),
                )
    for idx, feature in enumerate(gj["features"]):
        try:
            color = color_values[idx]
        except (ValueError):
            print(f"warning no rate found for index: {feature['properties']['FaultID']} faultname: `{feature['properties']['FaultName']}`")
            color = 'cyan'
        feature["properties"]["style"] = {
            "color": color,
            "weight": 1,
            "fillColor": color,
            "fillOpacity": 1,
        }
    scale = height / max(values)
    return show_3d(gj, -1, scale)

In [196]:
barchart_map(participation, "participation")

Map3DWidget(data=[{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'FaultID': 0, …

In [197]:
barchart_map(participationCrustal, "participation")

Map3DWidget(data=[{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'FaultID': 6, …

In [77]:
#show_3d(ruptures_only)

In [42]:
def falls_in(gj, lat_min, lat_max, lon_min, lon_max):
    p = gj['features'][0]['geometry']['coordinates'][0][0]
    lon = p[0]
    lat = p[1]
    return lat <= lat_max and lat >= lat_min and lon <= lon_max and lon >= lon_min

def cc_in (cc, lat_min, lat_max, lon_min, lon_max):
    lon = cc[0]
    lat = cc[1]
    return lon >= lon_min and lon <= lon_max and lat >= lat_min and lat <= lat_max

def feature_in(feature, lat_min, lat_max, lon_min, lon_max):
    geometry = feature['geometry']
    if geometry['type'] == 'Polygon':
        return all([cc_in(c, lat_min, lat_max, lon_min, lon_max) for c in geometry['coordinates'][0]]) 
    else:
        return False

def filter_gj(gj, lat_min, lat_max, lon_min, lon_max):
    return {'type':'FeatureCollection', 'features': [f for f in gj['features'] if feature_in(f, lat_min, lat_max, lon_min, lon_max)]}

def filter_list_of_gj(gjs, lat_min, lat_max, lon_min, lon_max):
    return [g for g in [filter_gj(gj, lat_min, lat_max, lon_min, lon_max) for gj in gjs] if len(g['features']) > 0]



    

In [33]:
#show_3d( bruceData[800:1300] + [bruce_hikurangi], -1)
#show_3d(bruce_puysegur,-1)

In [38]:
#features = [f for f in bruce_crustal if falls_in(f, -90, -43, 0, 170)]
#features = [f for f in features if f['features'][0]['geometry']['coordinates'][0][0][0] < 170]
#show_3d(features, -1)

In [9]:
hikurangi =filter_gj(bruce_hikurangi,  -90, -36, 0, 180)

In [10]:
len(hikurangi['features'])

78449

In [11]:
len(bruce_hikurangi['features'])

191800

In [13]:
#show_3d([hikurangi, opensha_hikurangi], -1)